In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import json
from tqdm import tqdm
import time
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
newDf = pd.read_csv("/content/drive/MyDrive/IR Project/Saved/dataframes/newDf.csv")

In [ ]:
newDf.head()

,Link,Name,Level:,Total:,Yield,Nutrition Info,ingredients_phrase,Method,ingredients,id
0,https://www.foodnetwork.com/recipes/rachael-ra...,Everything Lo Mein,Easy,30 min,4 servings,"{'Serving Size': '1 of 14 servings', 'Calories...","['3 rounded tablespoons hoisin sauce', '3 tabl...",['Mix together sauce ingredients and reserve.'...,"pad thai sauce, spaghetti, ginger snaps, pork ...",0
1,https://www.foodnetwork.com/recipes/food-netwo...,Bamya,Easy,1 hr 10 min,4 to 6 servings,"{'Serving Size': '1 of 8 servings', 'Calories'...","[""1 pound boneless lamb stew meat, cut into 1-...",['Mix the lamb with 1 teaspoon seven spice ble...,"tomato paste, garlic, beet juice, onion, olive...",1
2,https://www.foodnetwork.com/recipes/rachael-ra...,Spinach Fettuccine Primavera,Easy,27 min,4 to 6 servings,"{'Serving Size': '1 of 10 servings', 'Calories...","['Salt', '1 pound spinach fettuccine', '2 carr...",['Bring water for pasta to a boil. Salt the wa...,"green pea soup, garlic, spinach fettuccine, sh...",2
3,https://www.foodnetwork.com/recipes/katie-lee/...,Yogurt Wet Rub for Chicken,Easy,5 min,1 cup,"{'Serving Size': '1 of 2 servings', 'Calories'...","['1 cup Greek yogurt', '1 teaspoon turmeric', ...","['Mix the yogurt, turmeric, paprika, salt, gar...","garlic powder, greek yogurt, beet juice, turme...",3
4,https://www.foodnetwork.com/recipes/guy-fieri/...,Weird Spaghetti,Easy,2 hr 30 min,8 servings,"{'Serving Size': '1 of 16 servings', 'Calories...","['2 pounds ground beef, (80/20 lean muscle to ...","['In a large saute pan over medium heat, brown...","clove, gin, cumin, nutmeg, apple cider vinegar...",4


In [15]:
images_old = pd.read_csv("/content/drive/MyDrive/IR Project/Saved/images_dataset.csv")

In [16]:
images_dataset = pd.DataFrame()
for i in range(1,5):
    for j in range(1,5):
        link = '/content/drive/MyDrive/IR Project/Data Extraction Files (Image)/ImageLinks_'+str(i)+"_"+ str(j)+'.csv'
        curr_dataset = pd.read_csv(link)
        images_dataset = pd.concat([images_dataset,curr_dataset],axis = 0)
images_dataset

,Link,Image Link
0,https://www.foodnetwork.com/recipes/ragu-alla-...,https://food.fnr.sndimg.com/content/dam/images...
1,https://www.foodnetwork.com/recipes/damaris-ph...,https://food.fnr.sndimg.com/content/dam/images...
2,https://www.foodnetwork.com/recipes/parsley-an...,https://food.fnr.sndimg.com/content/dam/images...
3,https://www.foodnetwork.com/recipes/tomato-cuc...,https://food.fnr.sndimg.com/content/dam/images...
4,https://www.foodnetwork.com/recipes/food-netwo...,https://food.fnr.sndimg.com/content/dam/images...
...,...,...
1299,https://www.foodnetwork.com/recipes/berry-crum...,https://food.fnr.sndimg.com/content/dam/images...
1300,https://www.foodnetwork.com/recipes/food-netwo...,https://food.fnr.sndimg.com/content/dam/images...
1301,https://www.foodnetwork.com/recipes/tyler-flor...,https://food.fnr.sndimg.com/content/dam/images...
1302,https://www.foodnetwork.com/recipes/grilled-pi...,https://food.fnr.sndimg.com/content/dam/images...


In [61]:
# Merge the old and new
# 1. The ones not in Old
# 2, Ones in the old, keep the new one.
final_images = pd.concat([images_old,images_dataset],axis = 0)
final_images = final_images.drop_duplicates(subset = "Link", keep = "last")
final_images = final_images.reset_index(drop = True)

In [ ]:
final_images.to_csv('/content/drive/MyDrive/IR Project/Saved/dataframes/final_images_dataset.csv',index= False)

In [62]:
# redone = pd.read_csv('/content/drive/MyDrive/IR Project/Data Extraction Files (Image)/ImageLinks_redo.csv')
# final_final_images = pd.concat([final_images,redone],axis = 0)
# final_final_images = final_final_images.drop_duplicates(subset = "Link", keep = "last")
final_final_images = final_final_images.reset_index(drop = True)
final_final_images.to_csv('/content/drive/MyDrive/IR Project/Saved/dataframes/final_final_images_dataset.csv',index= False)
final_final_images.shape

(54853, 2)

In [ ]:
k = final_final_images['Image Link'].value_counts()[final_final_images['Image Link'].value_counts() == 2]
count = 0
for img_link in k.index:
    print(count, img_link)
    x = final_final_images[final_final_images['Image Link']== img_link]['Link']
    for l in x:
        print("\t",l)
    print()
    count+=1

In [59]:
# url = "https://www.foodnetwork.com/recipes/cinnamon-streusel-pizza-3361844"
# img = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOTwIHlN3AWA16CkcsjVWqiKzWchhmq4u4SPqsWCmeFHeCFSmGA_zM&usqp=CAE&s"

# url = "https://www.foodnetwork.com/recipes/sorbet-stuffed-cupcakes-recipe-2120702"
# img = "https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_860,ar_3:2/v1/img/recipes/49/48/82/pic3RD6C8.jpg"

# url = "https://www.foodnetwork.com/recipes/lemon-cream-pancake-puffs-recipe-2121001"
# img = "https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_860,ar_3:2/v1/img/recipes/49/48/84/picYBSBCi.jpg"

# url = "https://www.foodnetwork.com/recipes/sticky-sweet-drumsticks-recipe-2112519"
# img = "http://lh4.ggpht.com/_dfV_Eqz0Ypw/TUidWdXO6FI/AAAAAAAABlE/2vAARV2OpZ0/sweet%20sticky%20chicken%20drumsticks%5B4%5D.jpg?imgmax=800"

# url = "https://www.foodnetwork.com/recipes/spinach-salad-with-mandarin-oranges-and-walnuts-recipe-2109082"
# img = "https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_744,ar_5:4/v1/img/recipes/48/63/88/picN36Suu.jpg"

# url = "https://www.foodnetwork.com/recipes/jamie-oliver/sea-bass-with-fennel-and-olives-recipe-1910926"
# img = "https://i.pinimg.com/originals/bd/b3/5a/bdb35a197a156d36adc547d363429822.jpg"

# url = "https://www.foodnetwork.com/recipes/mac-cheese-muffins-recipe-2111753"
# img = "https://www.allrecipes.com/thmb/iflwt8VT43X-TlN8in13kmr_kHQ=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/3557771-easy-mac-and-cheese-muffins-Jenny-Anhold-4x3-1-c54cd382e8a243579ae50f4920e7f18d.jpg"

# url = "https://www.foodnetwork.com/recipes/green-chicken-and-cheese-enchiladas-1943525"
# img = "https://food.fnr.sndimg.com/content/dam/images/food/fullset/2009/4/5/0/CV1B01_29235_s4x3.jpg.rend.hgtvcom.826.620.suffix/1383164133134.jpeg"

url = "https://www.foodnetwork.com/recipes/nuoc-cham-dipping-sauce-3629526"
img = "https://food.fnr.sndimg.com/content/dam/images/food/fullset/2017/2/8/0/CCTIF306H_Summer-Rolls-with-Spicy-Peanut-Sauce-and-Nuoc-Cham-Dipping-Sauce_s4x3.jpg.rend.hgtvcom.826.620.suffix/1486567687744.jpeg"

final_final_images.loc[final_final_images['Link'] == url, 'Image Link'] = img

In [ ]:
# In the newer images dataset, which do not contain recipe name in the url: 
foodnetwork = "https://food.fnr.sndimg.com/content/dam/images/food/editorial/homepage/fn-feature.jpg.rend.hgtvcom.826.620.suffix/1474463768097.jpeg"
additionalScrape = []
for i, j in tqdm(images.iterrows(), total=images.shape[0]):
    link = j['Link'].split('/')[-1].lower()
    name = link.split('-')[:-1]
    try:
        imageLink = j['Image Link'].lower()
    except:
        additionalScrape.append(j['Link'])
    flag = 0
    for n in name:
        if n != 'recipe' and n in imageLink:
            flag = 1
    if flag == 0:
        additionalScrape.append(j['Link'])

100%|██████████| 48494/48494 [00:02<00:00, 19058.19it/s]


In [ ]:
len(additionalScrape)

12519

In [ ]:
# Of all the images in the newer images_dataset, which are pending from the newdf
final_scrape = []
available_links = set(images['Link'])
for i, j in tqdm(newDf.iterrows(), total=newDf.shape[0]): 
    if j['Link'] not in available_links:
        final_scrape.append(j['Link'])

100%|██████████| 54853/54853 [00:02<00:00, 23679.13it/s]


In [ ]:
len(final_scrape)

8345

In [ ]:
toScrape = final_scrape + additionalScrape

In [ ]:
print(len(toScrape))

20864


In [ ]:
f = open('/content/drive/MyDrive/IR Project/Saved/dataframes/redo_images.pickle', 'wb')
pickle.dump(toScrape,f)
f.close()

# Here.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import json
from tqdm import tqdm
import time
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
f = open('/content/drive/MyDrive/IR Project/Saved/dataframes/redo_images.pickle', 'rb')
final_scrape = pickle.load(f)
f.close()

In [ ]:
len(final_scrape)

20864

In [24]:
'''
Function to get the response for the specified url and parse the response using BeautifulSoup
'''
def get_soup(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    return soup

In [23]:
def extract(URL,errors):
    curr = ""
    try:
        soup = get_soup(URL)
        recipe_lead = soup.find('div', attrs = {'class' : "o-RecipeLead__m-RecipeMedia"})
        if recipe_lead is None:
            image_link = soup.find('img', attrs = {'class' : "kdp-poster__image"})
            if image_link is None: 
                image_link = soup.find('meta', attrs = {'property' : "og:image"})
                link = image_link['content']
                curr = link
            else:
                link = "https:" + image_link['src']
                curr = link
        else:
            image_link = recipe_lead.find('img', attrs = {'class' : "m-MediaBlock__a-Image a-Image"})
            link = "https:" + image_link['src']
            curr = link
    except Exception as e:
        errors.append(URL)
        curr = ""
    return curr

In [ ]:
def task(URLS, i ,j):
    image_links = []
    errors = []
    prev = "-"
    foodnetwork = "https://food.fnr.sndimg.com/content/dam/images/food/editorial/homepage/fn-feature.jpg.rend.hgtvcom.826.620.suffix/1474463768097.jpeg"
    for URL in tqdm(URLS):
        curr = extract(URL,errors)
        ctr = 0
        while prev == curr:
            if curr == foodnetwork:
                time.sleep(2)
                ctr+=1
                if(ctr>5):
                    break
                else:
                    curr = extract(URL,errors)
            else:
                time.sleep(2)
                curr = extract(URL,errors)
        image_links.append(curr)            
        prev = curr

    print("DONE!")
    Images = pd.DataFrame()
    Images['Link'] = URLS
    Images['Image Link'] = image_links
    Images.to_csv('/content/drive/MyDrive/IR Project/Data Extraction Files (Image)/ImageLinks_'+i+"_"+ j+'.csv',index = False)

    f = open('/content/drive/MyDrive/IR Project/Saved/dataframes/errors_'+i+"_"+ j+'.pickle', 'wb')
    pickle.dump(errors,f)
    f.close()

In [17]:
finaldf = pd.read_csv('/content/drive/MyDrive/IR Project/Saved/dataframes/final_images_dataset.csv')

In [18]:
redo_again = ["https://www.foodnetwork.com/recipes/southampton-clambake-in-a-can-11601901",
"https://www.foodnetwork.com/recipes/bobby-flay/everything-hash-brown-cake-with-pickled-red-onions-and-green-onion-creme-fraiche-2366132"]
k = finaldf['Image Link'].value_counts()[finaldf['Image Link'].value_counts()==3]
for img_link in k.index:
    # print(img_link)
    x = finaldf[finaldf['Image Link']== img_link]['Link']
    for l in x:
        # print("\t",l)
        redo_again.append(l)
    # print()
len(redo_again)

239

In [19]:
k = finaldf['Image Link'].value_counts()[finaldf['Image Link'].value_counts()==2]
for img_link in k.index:
    # print(img_link)
    x = finaldf[finaldf['Image Link']== img_link]['Link']
    for l in x:
        # print("\t",l)
        redo_again.append(l)
    # print()
len(redo_again)

1913

In [26]:
def task(URLS):
    image_links = []
    errors = []
    prev = "-"
    foodnetwork = "https://food.fnr.sndimg.com/content/dam/images/food/editorial/homepage/fn-feature.jpg.rend.hgtvcom.826.620.suffix/1474463768097.jpeg"
    for URL in tqdm(URLS):
        curr = extract(URL,errors)
        ctr = 0
        while prev == curr:
            ctr+=1
            if(ctr>5):
                break
            else:
                time.sleep(2)
                curr = extract(URL,errors)

        image_links.append(curr)            
        prev = curr

    print("DONE!")
    Images = pd.DataFrame()
    Images['Link'] = URLS
    Images['Image Link'] = image_links
    Images.to_csv('/content/drive/MyDrive/IR Project/Data Extraction Files (Image)/ImageLinks_redo.csv',index = False)

    f = open('/content/drive/MyDrive/IR Project/Saved/dataframes/errors_redo.pickle', 'wb')
    pickle.dump(errors,f)
    f.close()

In [29]:
import random
random.shuffle(redo_again)
task(redo_again)

100%|██████████| 1913/1913 [1:18:20<00:00,  2.46s/it]

DONE!


In [ ]:
from threading import Thread
all_recipe_links = final_scrape

fourth = int(len(all_recipe_links)/4)

all_recipe_links = all_recipe_links[:fourth]
j = "1"

# all_recipe_links = all_recipe_links[fourth: 2*fourth]
# j = "2"

# all_recipe_links = all_recipe_links[2*fourth:3*fourth]
# j = "3"

# all_recipe_links = all_recipe_links[3*fourth:]
# j = "4"

fourth = int(len(all_recipe_links)/4)

In [ ]:
t3 = Thread(target=task, args=[all_recipe_links[2*fourth:3*fourth],"3", j] )
t4 = Thread(target=task, args=[all_recipe_links[3*fourth:],"4", j] )
t2 = Thread(target=task, args=[all_recipe_links[fourth: 2*fourth],"2", j] )
t1 = Thread(target=task, args=[all_recipe_links[:fourth],"1", j] )

In [ ]:
t1.start()
t2.start()
t3.start()
t4.start()

  0%|          | 0/1304 [00:00<?, ?it/s]

  0%|          | 0/1304 [00:00<?, ?it/s]


  0%|          | 0/1304 [00:00<?, ?it/s]

In [ ]:
t1.join()
t2.join()
t3.join()
t4.join()

Streaming output truncated to the last 5000 lines.
 28%|██▊       | 359/1304 [1:04:44<1:26:25,  5.49s/it]

 28%|██▊       | 360/1304 [1:04:46<1:08:42,  4.37s/it]

 24%|██▎       | 309/1304 [1:04:52<4:39:36, 16.86s/it]


 30%|███       | 396/1304 [1:04:59<3:54:27, 15.49s/it]

 28%|██▊       | 362/1304 [1:05:04<1:57:35,  7.49s/it]

 28%|██▊       | 363/1304 [1:05:06<1:30:42,  5.78s/it]

 24%|██▍       | 310/1304 [1:05:09<4:37:02, 16.72s/it]


 30%|███       | 397/1304 [1:05:17<4:03:08, 16.08s/it]

 24%|██▍       | 311/1304 [1:05:26<4:38:59, 16.86s/it]

 26%|██▋       | 345/1304 [1:05:27<3:05:28, 11.60s/it]

 27%|██▋       | 346/1304 [1:05:29<2:21:51,  8.88s/it]


 24%|██▍       | 312/1304 [1:05:43<4:39:52, 16.93s/it]

 27%|██▋       | 349/1304 [1:05:50<1:41:39,  6.39s/it]


 24%|██▍       | 315/1304 [1:06:02<2:26:56,  8.91s/it]

 27%|██▋       | 350/1304 [1:06:07<2:32:28,  9.59s/it]


 24%|██▍       | 316/1304 [1:06:19<3:05:23, 11.26s/it]

 24%|██▍       | 317/1304 [1:06:21<2:18:00,  8.3

DONE!


 77%|███████▋  | 1002/1304 [3:22:25<56:07, 11.15s/it]

 77%|███████▋  | 1003/1304 [3:22:41<1:03:53, 12.73s/it]

 77%|███████▋  | 1004/1304 [3:22:58<1:09:50, 13.97s/it]

 77%|███████▋  | 1005/1304 [3:23:15<1:13:39, 14.78s/it]

 77%|███████▋  | 1006/1304 [3:23:31<1:15:22, 15.18s/it]

 77%|███████▋  | 1007/1304 [3:23:48<1:17:07, 15.58s/it]

 77%|███████▋  | 1008/1304 [3:24:04<1:17:57, 15.80s/it]

 77%|███████▋  | 1009/1304 [3:24:21<1:19:00, 16.07s/it]

 82%|████████▏ | 1063/1304 [3:24:37<44:03, 10.97s/it]

 78%|███████▊  | 1012/1304 [3:24:40<42:26,  8.72s/it]

 78%|███████▊  | 1015/1304 [3:24:58<28:48,  5.98s/it]

 78%|███████▊  | 1016/1304 [3:25:16<44:53,  9.35s/it]

 78%|███████▊  | 1017/1304 [3:25:32<54:53, 11.48s/it]

 78%|███████▊  | 1018/1304 [3:25:50<1:03:31, 13.33s/it]

 78%|███████▊  | 1021/1304 [3:26:11<38:05,  8.08s/it]

 83%|████████▎ | 1077/1304 [3:26:32<12:05,  3.20s/it]

 89%|████████▊ | 1157/1304 [3:26:35<41:13, 16.83s/it]

 89%|████████▉ | 1158/1304 [3:26:37<29:59, 12.32s

DONE!



100%|██████████| 1304/1304 [4:06:43<00:00, 11.35s/it]


DONE!


100%|██████████| 1304/1304 [4:19:31<00:00, 11.94s/it]


DONE!


Combining all Intermediate files to make 1 dataset.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import json
from tqdm import tqdm

In [ ]:
images_dataset = pd.DataFrame()
for i in range(1,4):
    for j in range(1,5):
        link = '/content/drive/MyDrive/IR Project/Data Extraction Files (Image)/ImageLinks_'+str(i)+"_"+ str(j)+'.csv'
        curr_dataset = pd.read_csv(link)
        images_dataset = pd.concat([images_dataset,curr_dataset],axis = 0)
images_dataset

,Link,Image Link
0,https://www.foodnetwork.com/recipes/rachael-ra...,https://food.fnr.sndimg.com/content/dam/images...
1,https://www.foodnetwork.com/recipes/guy-fieri/...,https://food.fnr.sndimg.com/content/dam/images...
2,https://www.foodnetwork.com/recipes/lemon-made...,https://food.fnr.sndimg.com/content/dam/images...
3,https://www.foodnetwork.com/recipes/new-orlean...,https://food.fnr.sndimg.com/content/dam/images...
4,https://www.foodnetwork.com/recipes/ina-garten...,https://food.fnr.sndimg.com/content/dam/images...
...,...,...
3424,https://www.foodnetwork.com/recipes/katie-lee/...,https://food.fnr.sndimg.com/content/dam/images...
3425,https://www.foodnetwork.com/recipes/tyler-flor...,https://food.fnr.sndimg.com/content/dam/images...
3426,https://www.foodnetwork.com/recipes/dave-liebe...,https://food.fnr.sndimg.com/content/dam/images...
3427,https://www.foodnetwork.com/recipes/grilled-pi...,https://food.fnr.sndimg.com/content/dam/images...


In [ ]:
images_part1 = pd.read_csv('/content/drive/MyDrive/IR Project/Saved/dataframes/images_dataset_1')

In [ ]:
images_dataset = pd.concat([images_part1,images_dataset], axis = 0)

In [ ]:
images_dataset = images_dataset.drop_duplicates(keep = 'last')

In [ ]:
images_dataset.to_csv('/content/drive/MyDrive/IR Project/Saved/images_dataset.csv',index = False)

In [ ]:
images_dataset.shape

(48494, 2)